# R Workflow in Amazon SageMaker

**Summary:**

This sample Notebook demonstrates an end-to-end workflow for R in SageMaker, including Hyperparameter tuning and how to generate predictions using two methods:  (1) offline [Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html); and (2) [deploying the model](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html) as a SageMaker endpoint and making online inferences.  Additionally, we'll see how to move a workflow from notebook prototyping to production with automated pipelines, in this case the SageMaker Pipelines feature. 

Since the focus is on demonstrating SageMaker mechanics, the use case is a simple one:  we'll predict abalone age, which is measured by the number of rings in the shell. We'll use the public [abalone dataset](https://archive.ics.uci.edu/ml/datasets/abalone) hosted by [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). 

We will use two different libraries to interact with SageMaker:
- [`Reticulate` library](https://rstudio.github.io/reticulate/): provides an R interface to make SageMaker API calls [Amazon SageMaker Python SDK](https://sagemaker.readthedocs.io/en/latest/index.html). The `reticulate` package translates between R and Python objects, and Amazon SageMaker provides a serverless data science environment to train and deploy ML models at scale.
- [`paws` library](https://cran.r-project.org/web/packages/paws/index.html): provides an interface to make API calls to AWS services, similar to how [`boto3`](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) works. `boto3` is the Amazon Web Services (AWS) SDK for Python. It enables Python developers to create, configure, and manage AWS services, such as EC2 and S3. Boto provides an easy to use, object-oriented API, as well as low-level access to AWS services. `paws` provides the same capabilities in R.

Table of Contents:
- [Reticulating the Amazon SageMaker Python SDK](#Reticulating-the-Amazon-SageMaker-Python-SDK)
- [Creating and Accessing the Data Storage](#Creating-and-accessing-the-data-storage)
- [Downloading and Processing the Dataset](#Downloading-and-processing-the-dataset)
- [Preparing the Dataset for Model Training](#Preparing-the-dataset-for-model-training)
- [Hyperparameter Tuning for the XGBoost Model](#Hyperparameter-Tuning-for-the-XGBoost-Model)
    - [Using the paws Library to Interact with AWS Services and Get the Status of the Tuning Job](#Using-the-paws-Library-to-Interact-with-AWS-Services-and-Get-the-Status-of-the-Tuning-Job)
- [Option 1: Batch Transform](#Option-1:-Batch-Transform)
    - [Create a Model using the Best Training Job](#Create-a-Model-using-the-Best-Training-Job)
    - [Batch Transform using the Tuned Estimator](#Batch-Transform-using-the-Tuned-Estimator)
    - [Download the batch job output](#Download-the-batch-job-output)
- [Option 2: Generate Predictions from an Endpoint](#Option-2:-Generate-Predictions-from-an-Endpoint)
    - [Deploying the Tuner](#Deploying-the-Tuner)
    - [Generating Predictions with the Deployed Model](#Generating-Predictions-with-the-Deployed-Model)
    - [Deleting the Endpoint](#Deleting-the-Endpoint)
- [Automating the workflow with SageMaker Pipelines](#Automating-the-workflow-with-SageMaker-Pipelines)
    - [Pipeline parameters](#Pipeline-parameters)
    - [Creating pipeline steps](#Creating-pipeline-steps)
    - [Setting up and starting the pipeline](#Setting-up-and-starting-the-pipeline)
    - [Viewing the pipeline in the Studio UI](#Viewing-the-pipeline-in-the-Studio-UI)
    

### Reticulating the Amazon SageMaker Python SDK

First, load the `reticulate` library and import the high level `sagemaker` Python SDK. Once it is loaded, you can use the `$` notation in R instead of the `.` notation in Python to use available classes. 

In [ ]:
# Turn warnings off globally
options(warn=-1)

In [ ]:
# Install reticulate library and import sagemaker
library(reticulate)
sagemaker <- import('sagemaker')

### Creating and Accessing the Data Storage

The `Session` class provides operations for working with the following [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) resources with Amazon SageMaker:

* [S3](https://boto3.readthedocs.io/en/latest/reference/services/s3.html):  AWS object storage service.
* [SageMaker](https://boto3.readthedocs.io/en/latest/reference/services/sagemaker.html):  APIs covering most SageMaker functionality.
* [SageMakerRuntime](https://boto3.readthedocs.io/en/latest/reference/services/sagemaker-runtime.html):  APIs for SageMaker real time and asynchronous endpoints.  (There are also separate APIs for SageMaker Feature Store and Edge Manager.)

Let's create an [Amazon Simple Storage Service](https://aws.amazon.com/s3/) bucket for your data. 

In [ ]:
session <- sagemaker$Session()
bucket <- session$default_bucket()

**Note** - The `default_bucket` function creates a unique Amazon S3 bucket with the following name: 

`sagemaker-<aws-region-name>-<aws account number>`

An AWS IAM role provides granular permissions to use AWS services.  Here we specify the IAM role's [ARN](https://docs.aws.amazon.com/general/latest/gr/aws-arns-and-namespaces.html) to allow Amazon SageMaker to access the Amazon S3 bucket. You can use the same IAM role used to create this Notebook:

In [ ]:
role_arn <- sagemaker$get_execution_role()

### Downloading and Processing the Dataset

The model uses the [abalone dataset](https://archive.ics.uci.edu/ml/datasets/abalone) from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). First, download the data and start the [exploratory data analysis](https://en.wikipedia.org/wiki/Exploratory_data_analysis). Use tidyverse packages to read, plot, and transform the data into ML format for Amazon SageMaker:

In [ ]:
library(readr)
data_file <- 'http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data'
abalone <- read_csv(file = data_file, col_names = FALSE)
names(abalone) <- c('sex', 'length', 'diameter', 'height', 'whole_weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'rings')
head(abalone)

The output above shows that `sex` is a factor data type but is currently a character data type (F is Female, M is male, and I is infant). Change `sex` to a factor and view the statistical summary of the dataset:

In [ ]:
abalone$sex <- as.factor(abalone$sex)
summary(abalone)

The summary above shows that the minimum value for `height` is 0.

Visually explore which abalones have height equal to 0 by plotting the relationship between `rings` and `height` for each value of `sex`:

In [ ]:
library(ggplot2)
options(repr.plot.width = 5, repr.plot.height = 4) 
ggplot(abalone, aes(x = height, y = rings, color = sex)) + geom_point() + geom_jitter()

The plot shows multiple outliers: two infant abalones with a height of 0 and a few female and male abalones with greater heights than the rest. Let's filter out the two infant abalones with a height of 0.

In [ ]:
library(dplyr)
abalone <- abalone %>%
  filter(height != 0)

### Preparing the Dataset for Model Training

We'll create three datasets: one each for training, testing, and validation. First, convert `sex` into a [dummy variable](https://en.wikipedia.org/wiki/Dummy_variable_(statistics)) and move the target, `rings`, to the first column. The SageMaker XGBoost algorithm, which we'll use to train a model, requires the target to be in the first column of the dataset.  (In general SageMaker can work with any data format required by the specific algorithm or framework.)

In [ ]:
abalone <- abalone %>%
  mutate(female = as.integer(ifelse(sex == 'F', 1, 0)),
         male = as.integer(ifelse(sex == 'M', 1, 0)),
         infant = as.integer(ifelse(sex == 'I', 1, 0))) %>%
  select(-sex)
abalone <- abalone %>%
  select(rings:infant, length:shell_weight)
head(abalone)

Next, sample 70% of the data for training the ML algorithm. Split the remaining 30% into two halves, one for testing and one for validation:

In [ ]:
abalone_train <- abalone %>%
  sample_frac(size = 0.7)
abalone <- anti_join(abalone, abalone_train)
abalone_test <- abalone %>%
  sample_frac(size = 0.5)
abalone_valid <- anti_join(abalone, abalone_test)

Later in the notebook, we'll use Batch Transform and Endpoints to generate predictions in two different ways, and compare the results. The maximum number of rows that we can send to an endpoint for inference in one batch is 500 rows (Batch Transform is better suited to batch scoring at scale than Endpoints). We are going to reduce the number of rows for the test dataset to 500 and use this for batch and online inference for comparison. 

In [ ]:
num_predict_rows <- 500
abalone_test <- abalone_test[1:num_predict_rows, ]

Next, we'll upload the training and validation data to Amazon S3 so we can train the model. First, write the training and validation datasets to the local filesystem in .csv format:

In [ ]:
write_csv(abalone_train, 'abalone_train.csv', col_names = FALSE)
write_csv(abalone_valid, 'abalone_valid.csv', col_names = FALSE)

# Remove target from test
write_csv(abalone_test[-1], 'abalone_test.csv', col_names = FALSE)

Second, upload the two datasets to the Amazon S3 bucket:

In [ ]:
s3_train <- session$upload_data(path = 'abalone_train.csv', 
                                bucket = bucket, 
                                key_prefix = 'data')
s3_valid <- session$upload_data(path = 'abalone_valid.csv', 
                                bucket = bucket, 
                                key_prefix = 'data')

s3_test <- session$upload_data(path = 'abalone_test.csv', 
                                bucket = bucket, 
                                key_prefix = 'data')

Finally, define the Amazon S3 input types for the Amazon SageMaker algorithm:

In [ ]:
s3_train_input <- sagemaker$inputs$TrainingInput(s3_data = s3_train,
                                     content_type = 'csv')
s3_valid_input <- sagemaker$inputs$TrainingInput(s3_data = s3_valid,
                                     content_type = 'csv')

### Hyperparameter Tuning for the XGBoost Model

Amazon SageMaker algorithms are available via [Docker](https://www.docker.com/) containers. To train an [XGBoost](https://en.wikipedia.org/wiki/Xgboost) model, specify the URL of the SageMaker XGBoost container, which is in [Amazon Elastic Container Registry](https://aws.amazon.com/ecr/) (Amazon ECR) for each AWS Region. We will use the `latest` version of the algorithm.

In [ ]:
container <- sagemaker$image_uris$retrieve(framework='xgboost', region= session$boto_region_name, version='latest')
cat('XGBoost Container Image URL: ', container)

Define an Amazon SageMaker [Estimator](http://sagemaker.readthedocs.io/en/latest/estimators.html), which can train any algorithm that has been containerized with Docker. There are many open source prebuilt containers for SageMaker for frameworks such as XGBoost, Scikit-learn, PyTorch, TensorFlow, etc.  You also can BYO container.

When creating the Estimator, use the following arguments:
* **image_uri** - The container image to use for training
* **role** - The Amazon SageMaker service role
* **train_instance_count** - The number of Amazon EC2 instances to use for training
* **train_instance_type** - The type of Amazon EC2 instance to use for training
* **train_volume_size** - The size in GB of the [Amazon Elastic Block Store](https://aws.amazon.com/ebs/) (Amazon EBS) volume to use for storing input data during training
* **train_max_run** - The timeout in seconds for training
* **input_mode** - The input mode that the algorithm supports
* **output_path** - The Amazon S3 location for saving the training results (model artifacts and output files)
* **output_kms_key** - The [AWS Key Management Service](https://aws.amazon.com/kms/) (AWS KMS) key for encrypting the training output
* **base_job_name** - The prefix for the name of the training job
* **sagemaker_session** - The Session object that manages interactions with Amazon SageMaker API

In [ ]:
s3_output <- paste0('s3://', bucket, '/output')
estimator <- sagemaker$estimator$Estimator(image_uri = container,
                                           role = role_arn,
                                           train_instance_count = 1L,
                                           train_instance_type = 'ml.m5.4xlarge',
                                           train_volume_size = 30L,
                                           train_max_run = 3600L,
                                           input_mode = 'File',
                                           output_path = s3_output,
                                           output_kms_key = NULL,
                                           base_job_name = NULL,
                                           sagemaker_session = NULL)

**Note** - The equivalent to `None` in Python is `NULL` in R.

Next, we specify the [XGBoost hyperparameters](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html) for the estimator, and also define the ranges of hyperparameters that we want to use for [SageMaker Automatic Model Tuning](https://sagemaker.readthedocs.io/en/stable/tuner.html). You can find the list of [Tunable Hyperparamters for the XGBoost algorithm here](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html). 

In addition, you need to specify the tuning evaluation metric. XGboost allows one of these nine objectives to be used (for the description of these objectives visit ["Tune an XGBoost Model"](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html) page) :

- validation:accuracy
- validation:auc
- validation:error
- validation:f1
- validation:logloss
- validation:mae
- validation:map
- validation:merror
- validation:mlogloss
- validation:mse
- validation:ndcg
- validation:rmse	

In this case, since this is a regression problem, we select `validation:rmse` as the tuning objective.

For tuning the hyperparameters you also need to specify the type and range of hyperparameters to be tuned. You can specify either a `ContinuousParameter` or an `IntegerParameter`, as outlined in the documentation. In addition, the algorithm documentation provides suggestions for the hyperparameter ranges.


Once the Estimator and its hyperparameters and tunable hyperparameter ranges are specified, you can create a `HyperparameterTuner` (tuner) object. You can train (or fit) that tuner, which will conduct the tuning and identify the best model in relation to the tuning objective. You can then generate predictions using the model with Batch Transform, or by deploying the model as an endpoint and using it for online inference.

In [ ]:
# Set Hyperparameters
estimator$set_hyperparameters(eval_metric='rmse',
                              objective='reg:linear',
                              num_round=100L,
                              rate_drop=0.3,
                              tweedie_variance_power=1.4)

In [ ]:
# Set Hyperparameter Ranges
hyperparameter_ranges <- list('eta' = sagemaker$parameter$ContinuousParameter(0,1),
                              'min_child_weight'= sagemaker$parameter$ContinuousParameter(0,10),
                              'alpha'= sagemaker$parameter$ContinuousParameter(0,2),
                              'max_depth'= sagemaker$parameter$IntegerParameter(0L,10L))

In [ ]:
# Set the tuning objective to RMSE
objective_metric_name <- 'validation:rmse'

The `HyperparameterTuner` accepts multiple parameters. A short list of these parameters are described below. For the complete list and more details you can visit the [`HyperparameterTuner` Documentation](https://sagemaker.readthedocs.io/en/stable/tuner.html#hyperparametertuner) :

- **estimator** (sagemaker.estimator.EstimatorBase) – An estimator object that has been initialized with the desired configuration. There does not need to be a training job associated with this instance.
- **objective_metric_name** (str) – Name of the metric for evaluating training jobs.
- **hyperparameter_ranges** (dict[str, sagemaker.parameter.ParameterRange]) – Dictionary of parameter ranges. These parameter ranges can be one of three types: Continuous, Integer, or Categorical. 
- **objective_type** (str) – The type of the objective metric for evaluating training jobs. This value can be either ‘Minimize’ or ‘Maximize’ (default: ‘Maximize’).
- **max_jobs** (int) – Maximum total number of training jobs to start for the hyperparameter tuning job (default: 1).
- **max_parallel_jobs** (int) – Maximum number of parallel training jobs to start (default: 1).

Here we've set the maximum jobs as 12, with maximum parallelization of 4 training jobs at a time.  Accordingly, the tuning job will run a series of 12/4 = 3 rounds of 4 concurrent training jobs.  The default tuning strategy, Bayesian Optimization, performs best if parallel jobs are iteratively run in several rounds so the overall 'meta-model' can make progress focusing in on the optimal hyperparameter ranges.  

In [ ]:
# Create a hyperparamter tuner
tuner <- sagemaker$tuner$HyperparameterTuner(estimator,
                                             objective_metric_name,
                                             hyperparameter_ranges,
                                             objective_type='Minimize',
                                             max_jobs=12L,
                                             max_parallel_jobs=4L)

# Create a tuning job name
job_name <- paste('sagemaker-tune-xgboost', format(Sys.time(), '%H-%M-%S'), sep = '-')

# Define the data channels for train and validation datasets
input_data <- list('train' = s3_train_input,
                   'validation' = s3_valid_input)


To start the tuning job, simply call the `fit` method of the `HyperparameterTuner` object.  

In [ ]:
tuner$fit(inputs = input_data, job_name = job_name, wait=FALSE)

#### Using the `boto3` SDK to Interact with AWS Services and Get the Status of the Tuning Job

With [`boto3` Python SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) you can create, configure, and manage AWS services, such as Amazon S3, Amazon SageMaker and other AWS services. The SDK provides an object-oriented API as well as low-level access to AWS services. Using `reticulate` library, you can leverage this SDK in R. 

Since running a tuning job may take a while, we are going to use `boto3` to get the status of the tuning job using `sagemaker$describe_hyper_parameter_tuning_job`.

In [ ]:
boto3_r <- import('boto3')

In [ ]:
# Create a paws SageMaker session
sm <- boto3_r$client('sagemaker')

Periodically check the tuning job status by re-running the cell below.  Eventually the status should become `Completed` with 12 'Succeeded' models after about 10 minutes of total job time.  You also can view the job status in the SageMaker console under **Hyperparameter tuning jobs**, which appears underneath the **Training** drop down.

In [ ]:
# Get the status of the tuning job
status <- sm$describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner$latest_tuning_job$job_name)

cat('Hyperparameter Tuning Job Name: ', job_name,'\n')
cat('Hyperparameter Tuning Job Status: ', status$HyperParameterTuningJobStatus,'\n')
cat('Succeeded Models:', status$ObjectiveStatusCounters$Succeeded,'\n')
cat('InProgress Models:', status$ObjectiveStatusCounters$Pending,'\n')
cat('Failed Models:', status$ObjectiveStatusCounters$Failed,'\n')


In [ ]:
# Print best training hyperparamters
status$BestTrainingJob$TunedHyperParameters

In [ ]:
# Print Evaluation Metric
status$BestTrainingJob$FinalHyperParameterTuningJobObjectiveMetric

In [ ]:
# Name of the best training job model
status$BestTrainingJob$TrainingJobName

<hr>

## Option 1: Batch Transform

### Create a Model using the Best Training Job
This section demonstrates how to create a model with the best training job results from the tuning job, using the model artifacts saved in S3.

First, we create a model container, which requires the following parameters:
- **Image:** URL of the algorithm container 
- **ModelDataUrl:** Location of the model tar ball (model.tar.gz) on S3 that is saved by the tuning job

We can extract the **ModelDataUrl** by describing the best training job using `boto3` SDK and `describe_training_job()` method. [More details can be found here](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.describe_training_job).
 
Then we will create a model using this model container. We will use `paws` library and the `create_model` method. [Documentation of this method can be found here](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model). 

In [ ]:
# Describe best model from tuning to get the location of the model artifact in S3
model_artifact <- sm$describe_training_job(
    TrainingJobName = status$BestTrainingJob$TrainingJobName
)$ModelArtifacts$S3ModelArtifacts

model_artifact

In [ ]:
# Create a model container wrapper
model_container <- list(
    "Image"= container,
    "ModelDataUrl" = model_artifact
    )

In [ ]:
# Create a SageMaker Model object

model_name <- paste('sagemaker-model-xgboost', format(Sys.time(), '%H-%M-%S'), sep = '-')

best_model <- sm$create_model(
    ModelName = model_name,
    PrimaryContainer = model_container,
    ExecutionRoleArn = role_arn
)

### Batch Transform using the Tuned Estimator

In many situations, using an endpoint for model deployment is not the best option, especially when the goal is not to generate online real-time predictions, but rather to generate predictions offline on a large stored dataset. In these situations, using Batch Transform may be more efficient and appropriate.

This section of the notebook explains how to set up the Batch Transform Job and generate predictions.

To do this, first we need to define the batch input data path on S3, and also where to save the generated predictions in S3.

In [ ]:
# Define S3 path for Test data and output path

s3_test_url <- paste('s3:/',bucket,'data','abalone_test.csv', sep = '/')
output_path <- paste('s3:/',bucket,'output/batch_transform_output',job_name, sep = '/')

Then we instantiate a `Transformer` object. [Transformers](https://sagemaker.readthedocs.io/en/stable/transformer.html#transformer) take multiple parameters, including the following. For more details and the complete list visit the [documentation page](https://sagemaker.readthedocs.io/en/stable/transformer.html#transformer).

- **model_name** (str) – Name of the SageMaker Model being used for the transform job.
- **instance_count** (int) – Number of EC2 instances to use.
- **instance_type** (str) – Type of EC2 instance to use, for example, ‘ml.c5.xlarge’.

- **output_path** (str) – S3 location for saving the transform job results. If not specified, results are stored to a default bucket.

- **base_transform_job_name** (str) – Prefix for the transform job when the transform() method launches. If not specified, a default prefix will be generated based on the training image name that was used to train the model associated with the transform job.

- **sagemaker_session** (sagemaker.session.Session) – Session object which manages interactions with Amazon SageMaker APIs and any other AWS services needed. If not specified, the estimator creates one using the default AWS configuration chain.

Once we instantiate a `Transformer` object, we can transform the batch input data.

In [ ]:
# Instantiate a SageMaker Transformer
transformer <- sagemaker$transformer$Transformer(
    model_name = model_name,
    instance_count=1L,
    instance_type='ml.m5.4xlarge',
    output_path=output_path,
    base_transform_job_name='R-Transformer',
    sagemaker_session=session)

In [ ]:
# Tranform the test data and wait until the task completes
transformer$transform(s3_test_url)
transformer$wait()

In [ ]:
# Get the status of Batch Transform
sm$describe_transform_job(TransformJobName = transformer$latest_transform_job$job_name)$TransformJobStatus


### Download the batch job output

The next step is to download the Batch Transform job results from where they are stored in S3.

In [ ]:
sagemaker$s3$S3Downloader$download(paste(output_path,"abalone_test.csv.out",sep = '/'),
                          "batch_output")

In [ ]:
# Read the batch csv from sagemaker local files
library(readr)
predictions <- read_csv(file = 'batch_output/abalone_test.csv.out', col_names = 'predicted_rings')
head(predictions)

Column-bind the predicted rings to the test data:

In [ ]:
# Concatenate predictions and test for comparison
abalone_predictions <- cbind(predicted_rings = predictions, 
                      abalone_test)
# Convert predictions to Integer
abalone_predictions$predicted_rings = as.integer(abalone_predictions$predicted_rings);
head(abalone_predictions)

In [ ]:
# Define a function to calculate RMSE
rmse <- function(m, o){
  sqrt(mean((m - o)^2))
}

In [ ]:
# Calucalte RMSE
abalone_rmse <- rmse(abalone_predictions$rings, abalone_predictions$predicted_rings)
cat('RMSE for Batch Transform: ', round(abalone_rmse, digits = 2))

<hr>

## Option 2: Generate Predictions from an Endpoint
### Deploying the Tuner

This section walks you through the endpoint deployment process for the tuned/trained model. We will then use the deployed model (as an endpoint) to make predictions using the test data. Deploying the model as as endpoint is suitable for cases where you need to make online predictions in response to data streaming in, for example from a consumer-facing app. For making predictions using offline batch data, the preferred method is using Batch Transform, which was demonstrated in the previous section.

Amazon SageMaker lets you [deploy your model](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html) by providing an endpoint that consumers can invoke by a secure and simple API call using an HTTPS request. Let's deploy our trained model to a `ml.t2.medium` instance. This will take a couple of minutes or less.

In [ ]:
model_endpoint <- tuner$deploy(initial_instance_count = 1L,
                                   instance_type = 'ml.t2.medium')

### Generating Predictions with the Deployed Model

Use the test data to generate predictions. Pass comma-separated text to be serialized into JSON format by specifying `text/csv` and `csv_serializer` for the endpoint:

In [ ]:
model_endpoint$serializer <- sagemaker$serializers$CSVSerializer(content_type='text/csv')

Remove the target column and convert the dataframe to a matrix with no column names:

In [ ]:
test_sample <- as.matrix(abalone_test[-1])
dimnames(test_sample)[[2]] <- NULL

Generate predictions from the endpoint and convert the returned comma-separated string:

In [ ]:
library(stringr)
predictions_ep <- model_endpoint$predict(test_sample)
predictions_ep <- str_split(predictions_ep, pattern = ',', simplify = TRUE)
predictions_ep <- as.integer(unlist(predictions_ep))

Column-bind the predicted ring numbers to those in the test data:

In [ ]:
# Convert predictions to Integer
abalone_predictions_ep <- cbind(predicted_rings = predictions_ep, 
                      abalone_test)
# abalone_predictions = as.integer(abalone_predictions)
head(abalone_predictions_ep)

In [ ]:
# Calucalte RMSE
abalone_rmse_ep <- rmse(abalone_predictions_ep$rings, abalone_predictions_ep$predicted_rings)
cat('RMSE for Endpoint 500-Row Prediction: ', round(abalone_rmse_ep, digits = 2))

### Deleting the Endpoint

When you're done with the model, delete the endpoint to avoid incurring deployment costs:

In [ ]:
session$delete_endpoint(model_endpoint$endpoint)

<hr>

## Automating the workflow with SageMaker Pipelines

In the previous sections of this notebook, we prototyped various steps of a project within the notebook itself, with some steps being run on external SageMaker resources (model tuning, batch transform, hosted endpoints). Notebooks are great for prototyping, but generally are not used in production-ready machine learning pipelines.

A very simple pipeline in SageMaker includes processing the dataset to get it reading for training, performing the actual training, and then using the model to perform some form of inference such as batch prediction. We'll use SageMaker Pipelines to automate some of these steps, keeping the pipeline simple for now: it easily can be extended into a far more complex pipeline with conditional steps and more.

### Pipeline parameters

Before we begin to create the pipeline itself, we should think about how to parameterize it. For example, we may use different instance types for different purposes, such as CPU-based types for batch scoring and GPU-based or more powerful types for model training. These are all "knobs" of the pipeline that we can parameterize. Parameterizing enables custom pipeline executions and schedules without having to modify the pipeline definition.

In the next cell several different pipeline parameters are specified. These include input data location (enabling future pipeline runs on different data slices without changing any code), and instance types and counts. Many others are possible.

In [ ]:
# input data
train_input_data_slice = sagemaker$workflow$parameters$ParameterString(name="TrainInputData", default_value=s3_train)
validation_input_data_slice = sagemaker$workflow$parameters$ParameterString(name="ValidationInputData", default_value=s3_valid)

# training step parameters (default instance type is a compute-optimized, powerful c5)
training_instance_type = sagemaker$workflow$parameters$ParameterString(name="TrainingInstanceType", default_value="ml.c5.2xlarge")
training_instance_count = sagemaker$workflow$parameters$ParameterInteger(name="TrainingInstanceCount", default_value=1L)

# batch inference step parameters (default instance type is a general purpose m5)
batch_instance_type = sagemaker$workflow$parameters$ParameterString(name="BatchInstanceType", default_value="ml.m5.4xlarge")
batch_instance_count = sagemaker$workflow$parameters$ParameterInteger(name="BatchInstanceCount", default_value=1L)

### Creating pipeline steps

The following code sets up a pipeline step for a training job.  The training step simply wraps the estimator created above, making it easy to move from prototyping to pipelines with a minimal learning curve.  Note that instead of a `TrainingStep`, we could have used a `TuningStep` incorporating Automatic Model Tuning similar to the code above (we're doing one training job here to shorten the time required to run this notebook).

In [ ]:
pipeline_estimator <- sagemaker$estimator$Estimator(image_uri = container,
                                                    role = role_arn,
                                                    instance_count = training_instance_count,
                                                    instance_type = training_instance_type,
                                                    volume_size = 30L,
                                                    train_max_run = 3600L,
                                                    input_mode = 'File',
                                                    output_path = s3_output,
                                                    output_kms_key = NULL,
                                                    base_job_name = NULL,
                                                    sagemaker_session = NULL)

pipeline_estimator$set_hyperparameters(eval_metric='rmse',
                                       objective='reg:linear',
                                       num_round=100L,
                                       rate_drop=0.3,
                                       tweedie_variance_power=1.4)

input_data <- list('train' = sagemaker$inputs$TrainingInput(s3_data = train_input_data_slice, content_type = 'csv'),
                   'validation' = sagemaker$inputs$TrainingInput(s3_data = validation_input_data_slice, content_type = 'csv'))

step_train <- sagemaker$workflow$steps$TrainingStep(
                name = "R-XGBoost-Train",
                estimator = pipeline_estimator,
                inputs = input_data)

We'll also create a SageMaker Model object to wrap the model artifact, and associate it with a separate SageMaker prebuilt XGBoost container to potentially use later for inference. For example, the Model could be used with SageMaker Batch Transform.

In [ ]:
pipeline_model <- sagemaker$model$Model(
                    image_uri = container,
                    model_data = step_train$properties$ModelArtifacts$S3ModelArtifacts,
                    sagemaker_session = session,
                    role = role_arn
                  )

model_hw_inputs <- sagemaker$inputs$CreateModelInput(instance_type = "ml.m5.large")

step_create_model <- sagemaker$workflow$steps$CreateModelStep(
                        name = "R-XGBoost-Create-Model",
                        model = pipeline_model,
                        inputs = model_hw_inputs
                    )

As a further step, we also can register the model in the SageMaker Model Registry for improved model governance. For example, with Model Registry you can catalog models for production, manage model versions, and associate metadata with models. Model Registry also can be used as part of a CI/CD workflow for model deployment to SageMaker Endpoints. By default, each time this pipeline is executed, a new model version will be registered.

In [ ]:
step_register <- sagemaker$workflow$step_collections$RegisterModel(
                    name = "R-XGBoost-Register-Model",
                    estimator = estimator,
                    model_data = step_train$properties$ModelArtifacts$S3ModelArtifacts,
                    content_types = c("text/csv", "text/csv"),
                    response_types = c("text/csv", "text/csv"),
                    inference_instances = c("ml.c5.xlarge", "ml.m5.xlarge"),
                    transform_instances = c("ml.c5.xlarge", "ml.m5.xlarge"),
                    model_package_group_name = "R-XGBoost-ModelPackageGroup",
                    image_uri = container
                  )

The final step in this pipeline is offline, batch scoring (inference) using the SageMaker Batch Transform feature. The inputs to this step will be the model trained in the pipeline training step, and the test data. 

In [ ]:
pipeline_transformer <- sagemaker$transformer$Transformer(
                            model_name = step_create_model$properties$ModelName,
                            instance_count = batch_instance_count,
                            instance_type = batch_instance_type,
                            output_path = output_path,
                            sagemaker_session = session
                        )

step_transform <- sagemaker$workflow$steps$TransformStep(
                        name = "R-XGBoost-Transform", 
                        transformer = pipeline_transformer, 
                        inputs = sagemaker$inputs$TransformInput(data = s3_test_url)
                    )

### Setting up and starting the pipeline

With all of the pipeline steps now defined, we can define the pipeline itself as a Pipeline object comprising a series of those steps. Parallel and conditional steps also are possible. The pipeline's directed acyclic graph (DAG) structure is inferred from the specified inputs and outputs, there is no need to explicitly specify the DAG or whether steps are parallel or in series.

In [ ]:
pipeline <- sagemaker$workflow$pipeline$Pipeline(
              name = 'R-XGBoost',
              parameters = c(train_input_data_slice,
                             validation_input_data_slice,
                             training_instance_type,
                             training_instance_count,
                             batch_instance_type,
                             batch_instance_count),
              steps = c(step_train, step_create_model, step_register, step_transform)
            )

We can quickly do a sanity check on the pipeline definition:

In [ ]:
pipeline$definition()

Next we upsert the pipeline definition. Using the upsert method allows us to modify the pipeline definition as needed for each execution. 

In [ ]:
pipeline$upsert(role_arn = role_arn)

A pipeline execution can be started with the `start` method.

In [ ]:
execution <- pipeline$start()

We can now confirm that the pipeline is executing. In the log output below, confirm that PipelineExecutionStatus is `Executing`.

In [ ]:
execution$describe()

Typically this pipeline should take about 10 minutes to complete. We can wait for completion by invoking wait(). After execution is complete, we can list the status of the pipeline steps.

In [ ]:
execution$wait()

In [ ]:
execution$list_steps()

### Viewing the pipeline in the Studio UI

#### Pipeline details

After the pipeline has started, we can examine the progress of the pipeline in the Studio UI.  To do so,

- Click the tilted triangle icon in the left panel toolbar, and select **Pipelines** from the drop down menu.
- Double-click the name of your pipeline, which should be `R-XGBoost`.
- In the new pipeline tab that opens, you should see a table of pipeline executions.  Double-click the first execution.
- Another tab opens with pipeline execution details.  Under the **Graph** sub-tab, you should see a pipeline node graph.  Typically in progress nodes are blue, succeeded are green, and failed are red.
- Click any of the the pipeline graph nodes that have finished to see details such as input, output and logs (which saves the effort of switching to a different UI, CloudWatch Logs, to inspect logs).  
- The **Settings** sub-tab will have general metadata about the pipeline execution.

#### Experiments

By default, a pipeline execution also will generate a SageMaker Experiment for tracking the overall workflow.  To examine this:

- Click the tilted triangle icon in the left panel toolbar, and select **Experiments and trials** from the drop down menu.
- Double-click the name of your Experiment, which should be `r-xgboost`.
- In the Trials table that opens, right-click the first trial and select **Open in trial components list**.  
- A Trial Components List tab will open.  For this simple pipeline, it will only have a Training job and Transform job.  You can right-click them and select **Open in trial details** to view metadata about each job.  More advanced usage is possible as you'll see from all of the sub-tab options that are available, but for sake of simplicity were not used in this example.

#### Model Registry

Since we had a RegisterModel step in our pipeline, the model also was registered in the SageMaker Model Registry.  To view the registry:

- Click the tilted triangle icon in the left panel toolbar, and select **Model registry** from the drop down menu.
- Double-click the name of your model group, which should be `R-XGBoost-ModelPackageGroup`.
- A tab will open for the Model Package Group.  It will have a list of model versions (by default they are numbered starting from 1 for each pipeline execution), and a sub-tab for Settings for the overall group.  
- You can double-click the model version to open up a version tab with metadata about the model version.  Although we didn't make use of it, there is addtional functionality for tracking the metrics associated with the model, performing manual approvals for model deployment to an endpoint, etc.